In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import time
import seaborn as sns
import datetime
import zipfile
import os
import warnings
warnings.filterwarnings("ignore")
#import streamlit as st

def extract_zip(zip_file_path, extracted_dir):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_dir)

zip_file_path = "data/StocksAndSectors.zip"
extracted_dir = "data/extracted_files"
extract_zip(zip_file_path, extracted_dir)

path_address = os.listdir('data/extracted_files/StocksAndSectors')

path_address = ['../relative strength strategy/data/extracted_files/StocksAndSectors/' + i for i in path_address]

data = pd.read_csv(path_address[1],skiprows=5)

for i in path_address[2:]:
    temp_data = pd.read_csv(i,skiprows=5)
    data = pd.concat([data, temp_data], axis = 0)    

In [35]:
data = pd.read_excel('data/stocks_list.xlsx')

In [36]:
data = data.drop_duplicates()
data = data[data['Market Cap(Rs. Cr.)']>500]

In [37]:
# numeric_pattern = r'^\d+$'
# data = data[~data['Symbol'].str.match(numeric_pattern)]

In [38]:
data

,Stock Name,Symbol,Industry,Market Cap(Rs. Cr.)
1,Prevest Denpro Ltd,PREVEST.BO,Medical Equipment & Supplies,5.017254e+02
2,A.K.CAPITAL SERVICES LTD.,AKCAPIT.BO,Non Banking Financial Company (NBFC),5.018310e+02
3,HIGH ENERGY BATTERIES (INDIA) LTD.,HIGHENE.BO,Household Products,5.024232e+02
4,DOLPHIN OFFSHORE ENTERPRISES (INDIA) LTD.-$,DOLPHINOFF.BO,Offshore Support Solution Drilling,5.028209e+02
5,ASM TECHNOLOGIES LTD.,ASMTEC.BO,Computers - Software & Consulting,5.034150e+02
...,...,...,...,...
1479,ITC LTD.,ITC.BO,Diversified FMCG,5.766855e+05
1480,BHARTI AIRTEL LTD.,BHARTIARTL.BO,Telecom - Cellular & Fixed line services,6.053543e+05
1481,HINDUSTAN UNILEVER LTD.,HINDUNILVR.BO,Diversified FMCG,6.257664e+05
1482,TATA CONSULTANCY SERVICES LTD.,TCS.BO,Computers - Software & Consulting,1.372630e+06


In [39]:
categories = data['Industry'].value_counts()
categories = categories[categories>0]

In [40]:
categories

Industry
Pharmaceuticals                         83
Auto Components & Equipments            71
Specialty Chemicals                     57
Industrial Products                     52
Non Banking Financial Company (NBFC)    43
                                        ..
Consulting Services                      2
Industrial Gases                         2
E-Learning                               2
Power - Transmission                     2
Oil Storage & Transportation             1
Name: count, Length: 146, dtype: int64

In [41]:
len(categories.index)

146

In [42]:
data = data[data['Industry'].isin(categories.index)]

In [43]:
#data = data.drop(columns = data.columns[data.isna().sum()>1000])

In [44]:
#list(data[data['Industry'] == 'Paper & Paper Products'].Symbol.values + ".NS")

In [45]:
stocks_list = {}
market_cap = {}
for i in categories.index:
    stocks_list[i] = list(data[data['Industry'] == i].Symbol.values + ".NS")
    market_cap[i] = list(data[data['Industry'] == i].loc[:, 'Market Cap(Rs. Cr.)'].values)

In [46]:
stocks_list = {}
market_cap = {}
for i in categories.index:
    stocks_list[i] = list(data[data['Industry'] == i].Symbol.values)
    market_cap[i] = list(data[data['Industry'] == i].loc[:, 'Market Cap(Rs. Cr.)'].values)

In [47]:
def return_analysis(stocks_dict, market_cap_dict, sector):
    
    import datetime
    from datetime import datetime, timedelta
    
    s = yf.download(stocks_dict[sector])
    s = s['Adj Close']
    s = s.sort_index()
    
    f = s/s.shift(1)
    
    indices_df = f.T.mean().fillna(1).to_frame().rename(columns = {0:'daily_nav'})
    
    s = s.iloc[[-1, -6, -22, -64]].T
    
    
    a = yf.download(stocks_dict[sector], interval = '1m', start = datetime.now() - timedelta(minutes = 30))
    a = a['Adj Close']
    a = a.sort_index().max().to_frame()
    
    s.iloc[:, 0] = a
    
    s['week_ret'] = s.iloc[:,0]/s.iloc[:,1]
    s['month_ret'] = s.iloc[:,0]/s.iloc[:,2]
    s['quarter_ret'] = s.iloc[:,0]/s.iloc[:,3]
    
    market_cap_df = pd.DataFrame(market_cap_dict[sector],stocks_dict[sector])
    s = pd.concat([s, market_cap_df], axis = 1)
    s = s.dropna()
    s = s.rename(columns = {0:'market_cap'})
    
    s['weight'] = s['market_cap']/s['market_cap'].sum()
    s['weighted_weekly_ret'] = s['week_ret'] * s['weight']
    s['weighted_month_ret'] = s['month_ret'] * s['weight']
    s['weighted_quarter_ret'] = s['quarter_ret'] * s['weight']
    
    sector_index_ret = [sector, s['weighted_weekly_ret'].sum(), s['weighted_month_ret'].sum(), s['weighted_quarter_ret'].sum()]
    
    return(s, sector_index_ret, indices_df)

In [48]:
returns = return_analysis(stocks_list, market_cap, 'Paper & Paper Products')

[*********************100%%**********************]  12 of 12 completed
[*********************100%%**********************]  12 of 12 completed


2 Failed downloads:
['ANDHRAPAP.BO', 'NRAGRINDQ.BO']: Exception('%ticker%: No price data found, symbol may be delisted (1m 2024-01-10 14:42:49.269977 -> 2024-01-10 15:12:49+05:30)')


In [49]:
returns[0]

,2024-01-10 00:00:00,2024-01-03 00:00:00,2023-12-11 00:00:00,2023-10-09 00:00:00,week_ret,month_ret,quarter_ret,market_cap,weight,weighted_weekly_ret,weighted_month_ret,weighted_quarter_ret
CENTURYTEX.BO,1577.800049,1245.650024,1290.650024,1098.199951,1.266648,1.222485,1.436715,13502.230000,0.386490,0.489547,0.472478,0.555276
EMAMIPAP.BO,119.000000,122.099998,135.000000,129.699997,0.974611,0.881481,0.917502,719.031200,0.020582,0.020059,0.018142,0.018884
JKPAPER.BO,390.600006,398.100006,399.700012,380.450012,0.981161,0.977233,1.026679,6923.294798,0.198174,0.194440,0.193662,0.203461
KUANTUM.BO,174.850006,174.300003,184.699997,178.600006,1.003155,0.946670,0.979003,1499.196058,0.042913,0.043049,0.040625,0.042012
ORIENTPPR.BO,43.250000,42.900002,42.770000,48.770000,1.008158,1.011223,0.886816,917.915107,0.026275,0.026489,0.026569,0.023301
PAKKA.BO,235.000000,240.649994,244.050003,216.199997,0.976522,0.962917,1.086957,859.347871,0.024598,0.024021,0.023686,0.026737
SATIA.BO,135.899994,138.750000,137.550003,126.349998,0.979459,0.988004,1.075584,1391.500000,0.039831,0.039012,0.039353,0.042841
SESHAPAPER.BO,365.649994,353.000000,369.950012,368.149994,1.035836,0.988377,0.993209,2233.558000,0.063934,0.066225,0.063191,0.063500
TNPL.BO,295.549988,290.549988,288.950012,249.800003,1.017209,1.022841,1.183146,2071.819000,0.059304,0.060325,0.060659,0.070165
WSTCSTPAPR.BO,700.750000,718.299988,694.400024,679.549988,0.975567,1.009145,1.031197,4817.607000,0.137900,0.134531,0.139161,0.142202


In [50]:
def nifty_ret():
    
    import datetime
    from datetime import datetime, timedelta
    
    todays_date = datetime.now().date()
    start_date = todays_date - timedelta(days = 200)

    s = yf.download("^NSEI", start = start_date)
    s = s['Adj Close']
    s = s.sort_index()
    s = s.to_frame()

    s = s.iloc[[-1, -6, -22, -64]].T
    
    a = yf.download("^NSEI", interval = '1m', start = datetime.now() - timedelta(minutes = 30))
    a = a['Adj Close']
    a = a.sort_index().max()
    s.iloc[:, 0] = a
    
    s['week_ret'] = s.iloc[:,0]/s.iloc[:,1]
    s['month_ret'] = s.iloc[:,0]/s.iloc[:,2]
    s['quarter_ret'] = s.iloc[:,0]/s.iloc[:,3]
    
    returns = ["nifty_ret", s['week_ret'].sum(), s['month_ret'].sum(), s['quarter_ret'].sum()]
    
    return(returns)

In [51]:
def stocks_selector(stocks_dict, market_cap_dict, sector):
    
    df_sectors = return_analysis(stocks_dict, market_cap_dict, sector)
    
    df_rets = df_sectors[1]
    df_full_df = df_sectors[0]
    indices_returns_df = df_sectors[2]
    
    df_nifty = nifty_ret()
    
    sectors = []
    stocks = []
    rs_score = []
    
    if (df_rets[1]>df_nifty[1])&(df_rets[2]>df_nifty[2])&(df_rets[3]>df_nifty[3]):
        
        try:
            sectors.append(df_rets[0])

            df_full_df['position'] = np.where((df_full_df['week_ret'] > df_rets[1])&
                                              (df_full_df['month_ret'] > df_rets[2])&
                                              (df_full_df['quarter_ret'] > df_rets[3]),
                                              1, 0)

            df_full_df = df_full_df[df_full_df['position'] == 1]

            df_full_df['rs_score'] = (df_full_df['week_ret']/df_rets[1])-1+(df_full_df['month_ret']/df_rets[2])-1+(df_full_df['quarter_ret']/df_rets[3])-1+(df_rets[1]/df_nifty[1])-1+(df_rets[2]/df_nifty[2])-1+(df_rets[3]/df_nifty[3])-1                  

            stocks.append(list(df_full_df.index))
            rs_score.append(list(df_full_df['rs_score'].values))

            df = pd.concat([pd.DataFrame(sectors).T, pd.DataFrame(stocks).T, pd.DataFrame(rs_score).T], axis = 1)
            df = df.fillna(method = "ffill")
            
        except Exception as e:
            
            print('hello')
        
    else:
        df = 'NA'
            
    return(df_rets, df, indices_returns_df)

In [52]:
sector_analysis = []
sector_returns = []
sector_indices = {}

for i in stocks_list.keys():
    try:
        lists = stocks_selector(stocks_list, market_cap, i)
        sector_analysis.append(lists[1])
        sector_returns.append(lists[0])
        sector_indices[i] = lists[2]
    except Exception as e:
        print(e)
        print('empty')

[*********************100%%**********************]  83 of 83 completed


1 Failed download:
['RAJNISH.BO']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")



[*********************100%%**********************]  83 of 83 completed


16 Failed downloads:
['ALBERTDA.BO']: Exception('%ticker%: No price data found, symbol may be delisted (1m 2024-01-10 14:42:55.906543 -> 2024-01-10 15:12:55+05:30)')
['LINCOPH.BO', 'BLISSGVS.BO', 'MEDICO.BO', 'NEULANDLAB.BO', 'INDSWFTLAB.BO', 'PGHL.BO', 'NGLFINE.BO', 'ORCHIDPHAR.BO', 'CAPPL.BO', 'ERIS.BO']: Exception('%ticker%: No price data found, symbol may be delisted (1m 2024-01-10 14:42:55.906543 -> 2024-01-10 15:12:56+05:30)')
['HESTERBIO.BO', 'JAGSNPHARM.BO', 'THEMISMED.BO', 'INDOCO.BO', 'AJANTPHARM.BO']: Exception('%ticker%: No price data found, symbol may be delisted (1m 2024-01-10 14:42:55.906543 -> 2024-01-10 15:12:57+05:30)')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  71 of 71 completed
[*********************100%%**********************]  71 of 71 completed

...


[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  2 of 2 completed


1 Failed download:
['REFEX.BO']: Exception('%ticker%: No price data found, symbol may be delisted (1m 2024-01-10 14:47:11.746254 -> 2024-01-10 15:17:11+05:30)')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  2 of 2 completed


1 Failed download:
['MPSLTD.BO']: Exception('%ticker%: No price data found, symbol may be delisted (1m 2024-01-10 14:47:12.805156 -> 2024-01-10 15:17:12+05:30)')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
'numpy.float64' object has no attribute 'fillna'
empty


In [53]:
sector_returns

[['Pharmaceuticals',
  1.0052175917762345,
  1.1040083330468666,
  1.2006674647141846],
 ['Auto Components & Equipments',
  1.036015888603742,
  1.0782350819094264,
  1.196267073050962],
 ['Specialty Chemicals',
  0.9611630411958124,
  1.0482070556956748,
  1.1053311524102107],
 ['Industrial Products',
  1.0056885020399429,
  1.0509114869832326,
  1.0863077321870687],
 ['Non Banking Financial Company (NBFC)',
  1.0234825680967994,
  1.0406115417034272,
  1.0495744979493136],
 ['Civil Construction',
  1.032113981181103,
  1.057591367172536,
  1.198069526878469],
 ['Residential, Commercial Projects',
  1.1004991642414945,
  1.1636459761289166,
  1.5135058210302488],
 ['Iron & Steel Products',
  1.0469244284819788,
  1.077157294831398,
  1.2931561277306751],
 ['Computers - Software & Consulting',
  1.0104440019027923,
  1.0362567257754463,
  1.091448375262965],
 ['Cement & Cement Products',
  0.9817124440499299,
  1.0061364762281857,
  1.1657813866138809],
 ['Other Textile Products',
  1.

In [54]:
sector_returns_df = pd.DataFrame(sector_returns, columns = ['sectors', 'week_ret', 'monthly_ret', 'quarterly_ret'])
sector_returns_df = sector_returns_df.set_index('sectors')
sector_returns_df = (sector_returns_df-1)*100
sector_returns_df

,week_ret,monthly_ret,quarterly_ret
sectors,,,
Pharmaceuticals,0.521759,10.400833,20.066746
Auto Components & Equipments,3.601589,7.823508,19.626707
Specialty Chemicals,-3.883696,4.820706,10.533115
Industrial Products,0.568850,5.091149,8.630773
Non Banking Financial Company (NBFC),2.348257,4.061154,4.957450
...,...,...,...
Ratings,0.856320,1.732821,1.911031
Consulting Services,5.138168,17.832004,24.565339
Industrial Gases,1.030403,-3.716553,-6.238449


In [55]:
filtered_sector_analysis = []
for i in sector_analysis:
    if type(i) == str:
        pass
    else:
        filtered_sector_analysis.append(i)

In [56]:
final_df = pd.concat(filtered_sector_analysis).dropna()

In [57]:
final_df.columns = ['sector', 'stock_name', 'RS']

In [58]:
final_df

,sector,stock_name,RS
0,Pharmaceuticals,ANUHPHR.BO,0.272682
1,Pharmaceuticals,ASTRAZEN.BO,0.754576
2,Pharmaceuticals,AUROPHARMA.BO,0.221663
3,Pharmaceuticals,DCAL.BO,0.388486
4,Pharmaceuticals,GLAXO.BO,0.593495
...,...,...,...
0,Construction Vehicles,BEML.BO,0.571144
0,E-Retail/ E-Commerce,NYKAA.BO,0.404372
0,Ferro & Silica Manganese,POCL.BO,1.762670
0,Financial Products Distributor,ANANDRATHI.BO,0.454291


In [59]:
top_20_df = final_df#.nlargest(20, columns = 'RS')

In [60]:
import datetime
from datetime import datetime, timedelta

'''datetime.now() - timedelta(minutes = 15)'''

def stock_current_price(stock_ticker):
    a = yf.download(stock_ticker, interval = '1m', start = datetime.now() - timedelta(minutes = 30))
    a = a['Adj Close']
    a = a.sort_index().max()
    return a

In [61]:
sl = 0.85
tp = 1.3
capital = 300000

In [62]:
top_20_df['current_price'] = top_20_df['stock_name'].apply(lambda x:stock_current_price(x))
top_20_df['sl'], top_20_df['tp'] = top_20_df['current_price']*sl, top_20_df['current_price']*tp

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******


1 Failed download:
['INDOTECH.BO']: Exception('%ticker%: No price data found, symbol may be delisted (1m 2024-01-10 14:47:23.032260 -> 2024-01-10 15:17:23+05:30)')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******

In [63]:
#filter on stock price

top_20_df = top_20_df[top_20_df['current_price']<5000]

In [64]:
top_20_df = top_20_df.nlargest(20, columns = 'RS')

In [65]:
top_20_df = top_20_df.sort_values(by = 'current_price')

In [66]:
def buy_stocks(prices, capital):
    total_stocks = len(prices)
    
    equal_weight = capital / total_stocks
    
    stocks_to_buy = np.floor(equal_weight / prices)
    
    allocation = stocks_to_buy * prices
    
    remaining_cash = capital - np.sum(stocks_to_buy * prices)
    
    while remaining_cash > 0:
        
        idx = np.argmin(allocation)
        stocks_to_buy[idx] += 1
        remaining_cash -= prices[idx]
    
    return stocks_to_buy

prices = top_20_df['current_price'].values

stocks_to_buy = buy_stocks(prices, capital)
top_20_df['number_of_stocks'] = stocks_to_buy

In [67]:
top_20_df['allocation'] = (top_20_df['current_price'] * top_20_df['number_of_stocks'])*100/capital

In [68]:
top_20_df = top_20_df.sort_values(by = 'RS', ascending=False)

In [69]:
top_20_df.reset_index(drop = True, inplace = True)

In [70]:
top_20_df

,sector,stock_name,RS,current_price,sl,tp,number_of_stocks,allocation
0,"Gems, Jewellery And Watches",PCJEWELLER.BO,2.302247,64.690002,54.986502,84.097003,231.0,4.981130
1,Ferro & Silica Manganese,POCL.BO,1.762670,797.049988,677.492490,1036.164984,18.0,4.782300
2,Iron & Steel Products,GALLANTT.BO,1.753880,188.800003,160.480003,245.440004,79.0,4.971733
3,"Residential, Commercial Projects",SIGNATURE.BO,1.411396,1122.000000,953.700000,1458.600000,13.0,4.862000
4,Logistics Solution Provider,TIGERLOGS.BO,1.314439,833.000000,708.050000,1082.900000,18.0,4.998000
5,Pharmaceuticals,GUJTHEM.BO,1.310802,336.399994,285.939995,437.319992,44.0,4.933867
6,Logistics Solution Provider,NAVKARCORP.BO,1.183047,120.800003,102.680003,157.040004,124.0,4.993067
7,Telecom - Infrastructure,GTLINFRA.BO,1.158312,1.800000,1.530000,2.340000,8333.0,4.999800
8,Iron & Steel Products,SURYAROSNI.BO,1.141834,834.900024,709.665021,1085.370032,22.0,6.122600
9,Media & Entertainment,NETWORK18.BO,1.111556,119.370003,101.464502,155.181004,125.0,4.973750


In [71]:
top_20_df['allocation'].sum()

100.03964016270638

In [72]:
import sqlalchemy

In [73]:
engine = sqlalchemy.create_engine('sqlite:///daily_stocks.db')

In [74]:
top_20_df['date'] = pd.to_datetime(datetime.now().date())

In [75]:
top_20_df.to_sql('daily_stocks', engine, if_exists='append', index=False)

20

In [76]:
engine1 = sqlalchemy.create_engine('sqlite:///sectoral_indices.db')

In [109]:
for keys, values in sector_indices.items():
    
    date_indb = pd.read_sql(keys, engine1)['Date'].iloc[-1]
    
    if date_indb < values.index[-1]:
        
        values.loc[date_indb:].iloc[1:].to_sql(keys, engine1, if_exists='append', index=True)
        
    print(f'{keys} values uploaded from {date_indb}')

Pharmaceuticals values uploaded from 2024-01-10 00:00:00
Auto Components & Equipments values uploaded from 2024-01-10 00:00:00
Specialty Chemicals values uploaded from 2024-01-10 00:00:00
Industrial Products values uploaded from 2024-01-10 00:00:00
Non Banking Financial Company (NBFC) values uploaded from 2024-01-10 00:00:00
Civil Construction values uploaded from 2024-01-10 00:00:00
Residential, Commercial Projects values uploaded from 2024-01-10 00:00:00
Iron & Steel Products values uploaded from 2024-01-10 00:00:00
Computers - Software & Consulting values uploaded from 2024-01-10 00:00:00
Cement & Cement Products values uploaded from 2024-01-10 00:00:00
Other Textile Products values uploaded from 2024-01-10 00:00:00
IT Enabled Services values uploaded from 2024-01-10 00:00:00
Commodity Chemicals values uploaded from 2024-01-10 00:00:00
Heavy Electrical Equipment values uploaded from 2024-01-10 00:00:00
Private Sector Bank values uploaded from 2024-01-10 00:00:00
Pesticides & Agroche

Granites & Marbles values uploaded from 2024-01-10 00:00:00
Airport & Airport services values uploaded from 2024-01-10 00:00:00
Ratings values uploaded from 2024-01-10 00:00:00
Consulting Services values uploaded from 2024-01-10 00:00:00
Industrial Gases values uploaded from 2024-01-10 00:00:00
E-Learning values uploaded from 2024-01-10 00:00:00
Power - Transmission values uploaded from 2024-01-10 00:00:00


def exits(daily_stocks_dataframe = daily_stocks, portfolio_stocks_dataframe=portfolio_stocks):
    
#exits
    
    exits = []
    
    for index, rows in portfolio_stocks_dataframe.iterrows():
        
        stock_df = yf.download(rows['stock_name'], interval = '1m', start = datetime.now().date())
        
        if stock_df['High'].max() > rows['tp']:
            
            exits.append(rows['stock_name'])

        if stock_df['Low'].min() < rows['sl']:
            
            exits.append(rows['stock_name'])
            
    portfolio_stocks_dataframe = portfolio_stocks_dataframe[portfolio_stocks_dataframe['stock_name'] != exits]
    
    return portfolio_stocks_dataframe, exits

exits_df, exits_df = exits()

exits_df.to_sql('portfolio_stocks', if_exists = 'replace', con = engine, index = False)

def exits(daily_stocks_dataframe = daily_stocks, portfolio_stocks_dataframe=portfolio_stocks):
    
#exits
    
    exits = []
    
    for index, rows in portfolio_stocks_dataframe.iterrows():
        
        stock_df = yf.download(rows['stock_name'], interval = '1m', start = datetime.now().date())
        
        if stock_df['High'].max() > rows['tp']:
            
            exits.append(rows['stock_name'])

        if stock_df['Low'].min() < rows['sl']:
            
            exits.append(rows['stock_name'])
            
    portfolio_stocks_dataframe = portfolio_stocks_dataframe[portfolio_stocks_dataframe['stock_name'] != exits]
    
    return portfolio_stocks_dataframe

import pickle

output_path = r"..\relative strength strategy\final_datas"

with open(f"{output_path}\\sector_analysis.pkl", "wb") as f:
    pickle.dump(sector_analysis, f)

with open(f"{output_path}\\sector_returns.pkl", "wb") as f:
    pickle.dump(sector_returns, f)

with open(f"{output_path}\\sector_indices.pkl", "wb") as f:
    pickle.dump(sector_indices, f)